# CNNbasic Assignment#1


마크다운과 코드 속 `???` 를 채워주시면 됩니다!-!


- 제출자: 20기 황태연
- 제출 일자: 2023.09.05. (화)

# AlexNet

모델 구조 사진과 논문 사이트입니다.

- paper: http://www.cs.toronto.edu/~fritz/absps/imagenet.pdf

- dataset: http://image-net.org/challenges/LSVRC/2012/index#task

- Model architecture

<img src="01.png">

![model_architecture2](https://t1.daumcdn.net/cfile/tistory/99FEB93C5C80B5192E)

- The second convolutional layer takes as input the (response-normalized
and pooled) output of the first convolutional layer and filters it with 256 kernels of size 5 × 5 × 48.
The third, fourth, and fifth convolutional layers are connected to one another without any intervening
pooling or normalization layers. The third convolutional layer has 384 kernels of size 3 × 3 ×
256 connected to the (normalized, pooled) outputs of the second convolutional layer. The fourth
convolutional layer has 384 kernels of size 3 × 3 × 192 , and the fifth convolutional layer has 256
kernels of size 3 × 3 × 192. The fully-connected layers have 4096 neurons each.


### Naive Version
CONV_1 - POOL_1 - CONV_2 - POOL_2 - CONV_3 - CONV_4 - CONV_5 - POOL_3 - FC1 - FC2 - FC3 (->SOFTMAX)

### detailed
CONV_1(ReLU) - POOL_1 - CONV_2(ReLU) - POOL_2 - CONV_3(ReLU) - CONV_4(ReLU) - CONV_5(ReLU) - POOL_3 -(Flatten) FC1(ReLU) - FC2(ReLU) - FC3(->SOFTMAX)

# 논문 리뷰: ImageNet Classification with Deep Convolutional Neural Networks

# 0. ImageNet

- **ILSVRC(ImageNet Large Scale Visual Recognition Challenge)**: 이미지 인식 기술을 겨루는 대회
    - ILSVRC 2012년 대회에서 딥러닝에 기초한 AlexNet이 압도적으로 우승하면서 이후의 ILSVRC 대회의 주역은 딥러닝이 되었다.
    
- **이미지넷(ImageNet)**: 100만 장이 넘는 이미지를 담고 있는 데이터셋
<img src="02.png" width=70% height=70%>
(출처: https://opus.nci.org.au/display/DAE/Introduction+to+ImageNet)

- ILSVRC 대회에는 ImageNet의 데이터를 사용하는 대회로, 다음과 같은 시험 항목이 있다.  
    1. **Image Classification(이미지 분류)**
        - 가장 기본적인 시험 항목으로, 많은 연구자들이 참여한다.
        - 주어진 이미지를 1000개의 클래스 중 하나로 분류한다.
        - 분류 성능은 Top-1 Error와 Top-5 Error로 측정된다.
            - Top-1 Error: 모델이 예측한 클래스 중 실제 클래스와 일치하지 않는 것의 비율
            - Top-5 Error: 모델이 예측한 상위 5개의 클래스 중 실제 클래스가 포함되지 않은 것의 비율
    2. Object Detection(물체 검출)
        - 주어진 이미지에서 물체의 위치와 종류를 예측한다.

# 1. The Architecture

## 1.1. ReLU Nonlinearity

- **Saturating nonlinearities**: 비선형 함수 중에서, 입력 값이 크거나 작아질수록 출력 값이 포화(saturation)되는 함수
    - 예: sigmoid 함수(입력 값이 작을 때에는 0, 클 때에는 1에 가까워져서 값이 매우 크거나 작을 때에는 포화 상태에 빠져서 출력 값이 거의 변하지 않는다.)
- **Non-saturating nonlinearities**: 출력 값이 포화되지 않는 함수
    - 예: ReLU 함수
- 이 논문에서는 활성화 함수로 ReLU 함수를 사용하는 것이 훨씬 빠른 학습이 가능하다는 것을 보여준다.
<center><img src="03.png" width=30% height=30%></center>

- 위 그림은 CIFAR-10에 대한 훈련 데이터 오류율이다.
- 활성화 함수로 ReLU(실선)를 사용한 것이 tanh(점선)을 사용한 것보다 오류율 25%에 6배 빨리 도달하였다.

## 1.2. Training on Multiple GPUs

- AlexNet은 **두 개의 GTX 580 GPU**를 사용해서 학습한다.
- 각 GPU는 배치 사이즈의 절반에 해당하는 데이터를 나눠서 학습한다.
    - 이에 따라 GPU의 메모리 문제를 해결하고, 학습 시간을 단축할 수 있었다.
- **Cross-GPU parallelization**: AlexNet에서는 두 개의 GPU가 각각 학습을 하지만, **3번째 CNN 레이어**에서 **서로 소통**하여 각 GPU에서 계산한 **출력(feature map)을 절반씩 나눠**갖는다. 
    - Cross-GPU parallelization을 사용하여 top-1 오류율을 1.7%, top-5 오류율을 1.2% 감소시켰다.

## 1.3. Local Response Normalization

- 활성화 함수 ReLU는 너무 큰 input을 그대로 output으로 반환한다는 특징이 있다.
- 이런 큰 값은 Conv 계산이나 Pooling 연산에서 주변 값에 지나친 영향을 줄 가능성이 있다.
- 이를 막기 위해 Local Response Normalization을 이용한다.
$$b_{x, y}^{i} = a_{x, y}^{i} / \left( k + \alpha \sum_{j = max(0, i-n/2)}^{min(N-1, i+n/2)} (a_{x, y}^{j})^2\right)^{\beta}$$

($b_{x, y}^{i}$: LRN의 결과, $a_{x, y}^{i}$: 활성화 함수의 output, $i$: 현재 filter,  
$N$: filter의 총 개수, $n$: 정규화할 filter 수, $\alpha, \beta, k, n$: hyperparameters)

- 최근에는 LRN 대신 **Batch Normalization**을 사용한다.

## 1.4. Overlapping Pooling

- **Overlapping Pooling**: 인접한 Pooling 영역이 중첩되도록 설정하여 사용하는 방법
- 기존의 CNN을 사용한 모델의 Pooling layer에서는 영역이 겹치지 않도록 하였다.
- 이 논문에서는 Maxpooling을 진행할 때 서로 영역이 겹치도록 **stride: 2, kernel size: 3x3**으로 설정하였다.

## 1.5. Overall Architecture

- GPU 2개를 이용하는 AlexNet의 구조
<center><img src="04.png" width=70% height=70%></center>

- **주의!**: AlexNet과 관련된 여러 자료들이 input이 224 x 224 x 3이 아니라 **227 x 227 x 3**이 올바르다고 언급했다. 실제로 후자가 수학적으로 올바르다.

# 2. Reducing Overfitting

## 2.1. Data Augmentation

- 첫 번째 데이터 증강 방법: **수평 반전** & **crop**
<center><img src="05.png" width=50% height=50%></center>

- 1) 먼저 모든 이미지를 256x256으로 resize한다.
- 2) 수평 반전으로 이미지를 2배 증강한다.
- 3) 224x224로 random crop하여 32x32배 증강한다.
- 이렇게 총 이미지를 **2048배 증강**할 수 있다.
- 단, 추론 시에는 수평 반전과 네 모서리 및 중앙 crop으로 10개의 이미지를 사용한다.

- 두 번째 데이터 증강 방법: **Fancy PCA**
- **PCA(Principal Component Analysis)**: 데이터의 주성분(principal component)을 찾아내어 데이터의 차원을 축소하는 기술
    - 1) 공분산 행렬 구하기
    - 2) 고유값 분해 (고유값과 고유벡터를 각각 구한다.)
    - 3) 주성분 선택 (상위 k개의 고유값에 대한 고유벡터들로 데이터의 차원을 축소할 수 있다.)
- **Fancy PCA**: 이미지의 RGB 값에 PCA와 가우시안 랜덤 변수를 적용하여 이미지를 증강하는 기법
<center><img src="06.png" width=70% height=70%></center>

- Fancy PCA는 자연스러운 반면, Random Augmentation은 색에 대한 왜곡이 발생한다.

- 식은 다음과 같다.

$$I'_{xy} = [I_{xy}^{R}, I_{xy}^{G}, I_{xy}^{B}]^T + [\mathbf{p}_{1}, \mathbf{p}_{2}, \mathbf{p}_{3}][\alpha_{1} \lambda_{1}, \alpha_{2} \lambda_{2}, \alpha_{3} \lambda_{3}]^{T}$$

($I'_{xy}$: Fancy PCA로 생성된 이미지의 $xy$위치의 RGB값,  

$I_{xy}$: 원본 이미지의 $xy$위치의 RGB값,  

$\mathbf{p}_{i}$: 3x3 공분산 행렬의 고유벡터,

$\lambda_{i}$: 3x3 공분산 행렬의 고유값,

$\alpha_{i}$: 평균이 0, 표준편차가 0.1인 가우시안 랜덤 변수)

- 이 기술이 top-1 오류율을 1% 이상 감소시켰다.

## 2.2. Dropout

- Dropout는 앞의 노드와 그와 연결된 가중치를 제거하는 방식으로 작동한다.
- AlexNet에서는 첫 번째와 두 번째 FC 앞에서 Dropout을 적용하여 과적합을 방지했다.

### 참고 자료

1. AlexNet 논문: https://proceedings.neurips.cc/paper_files/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf
2. AlexNet Wikipedia: https://en.wikipedia.org/wiki/AlexNet
3. https://killerwhale0917.tistory.com/14
4. https://taeguu.tistory.com/29
5. https://daeun-computer-uneasy.tistory.com/33
6. Data Augmentation: https://learnopencv.com/understanding-alexnet/
7. Fancy PCA: https://pixelatedbrian.github.io/2018-04-29-fancy_pca/

# 과제

- 정확한 AlexNet의 Layer를 보여주는 그림은 다음과 같다.
<img src="07.png" width=70% height=70%>
(출처: https://medium.com/analytics-vidhya/concept-of-alexnet-convolutional-neural-network-6e73b4f9ee30)

## Convolution Layer

- 다음 식을 이용해서 **convolution layer의 output size**를 계산한다.
$$
OW = \frac{W+2P-FW}{S} + 1
$$
($OW$: Output Width, $W$: Width, $P$: Padding, $FW$: Filter Width, $S$: Stride)

- 다음 식을 이용해서 **convolution layer의 parameter 수**를 계산할 수 있다.

$$
(K \times C_{input} + B) \times C_{output}
$$
($K$: kernel size, $C$: number of channel, $B$: bias)

- 다음 함수는 **Convolution layer**의 output_size와 parameter 수를 출력한다.

In [19]:
def Conv2d(input_size, num_filters, filter_size, stride, padding):
    I = input_size # ex: (227, 227, 3)
    N = num_filters # ex: 96
    F = filter_size # ex: (11, 11)
    S = stride # ex: 4
    P = padding # ex: 0
    
    # output_size
    I_width = int(I[0])
    O_width = int((I_width + 2*P - F[0]) / S) + 1
    output_size = (O_width, O_width, N) # (output_width, output_width, output_channel), output_channel = num_of_filters
    
    # num_of_parameters
    I_channel = I[2]
    num_parameters = (F[0]*F[1] * I_channel + 1) * N
    
    print('Output Size:',output_size)
    print('Number of Parameters:', num_parameters)

## Max Pooling Layer

- 다음 식을 이용해서 **max pooling layer의 output_size**를 계산한다.
$$
OW = \frac{W - FW}{S} + 1
$$

- 다음 함수는 Max pooling의 output_size를 출력한다. max pooling layer에서는 학습하는 파라미터가 없다.

In [23]:
def MaxPool2d(input_size, filter_size, stride):
    I = input_size # ex: (55, 55, 96)
    F = filter_size # ex: (3, 3)
    S = stride # ex: 2
    
    I_width = int(I[0])
    O_width = int((I_width - F[0]) / S) + 1
    output_size = (O_width, O_width, I[2])
    
    print('Output Size:', output_size)
    print('Number of Parameters:', 0)

## Fully Connected Layer

- 다음 식을 이용하여 **fully connected layer의 parameter 수**를 계산할 수 있다.
$$
(I + 1) \times O
$$
($I$: Input size, $O$: Output size)

- 다음 함수는 **fully connected layer**의 parameter 수를 출력한다.

In [35]:
def Linear(input_size, output_size):
    num_parameters = (input_size + 1) * output_size
    print('Number of Parameters:', num_parameters)

## Layer 1 is a Convolution Layer_1

- **Input Image size**     `(227, 227, 3)`

- **Number of filters**     `96`

- **Filter size**     `(11, 11)`

- **Stride**     `4`

- **Layer 1 Output**     `(55, 55, 96)`

In [21]:
## TODO ##

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Conv_1 = Conv2d((227, 227, 3), 96, (11, 11), 4, 0)

Output Size: (55, 55, 96)
Number of Parameters: 34944


## Layer 2 is a Max Pooling_1 Followed by Convolution_1

- **Input**     `(54, 54, 96)`

- **Max pooling**     `(3, 3)`  

- **Pooling size** (overlapping)      `1` (filter size가 3x3이고 stride가 2이면 겹치는 구간의 너비는 1이다.)

    * overlapping 중첩되어서 지나감

- **Stride**     `2`

- **Layer 2 Output**     `(27, 27, 96)`
    
    * 차원은 변함 없음

In [25]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Max_pool_1 = MaxPool2d((55, 55, 96), (3, 3), 2)

Output Size: (27, 27, 96)
Number of Parameters: 0


## Layer 3 is a Convolution Layer_2

- **Input**     `(27, 27, 96)`

- **Number of filters**     `256`

- **Filter size**     `(5, 5)`

- **Stride**     `1`

- **padding**     `2`

- **Layer 3 Output**     `(27, 27, 256)`

In [27]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Conv_2 = Conv2d((27, 27, 96), 256, (5, 5), 1, 2)

Output Size: (27, 27, 256)
Number of Parameters: 614656


## Layer 4 is a Max Pooling_2 Followed by Convolution_2

- **Input**     `(27, 27, 256)`

- **Max pooling**     `(3, 3)`  

- **Pooling size**(overlapping)     `1`

- **Stride**     `2`

- **Layer 4 Output**     `(13, 13, 256)`

In [28]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Max_pool_2 = MaxPool2d((27, 27, 256), (3, 3), 2)

Output Size: (13, 13, 256)
Number of Parameters: 0


## Layer 5 is a Convolution Layer_3

- **Input**     `(13, 13, 256)`

- **Number of filters**     `384`

- **Filter size**     `(3, 3)`

- **Stride**     `1`

- **padding**     `1`

- **Layer 5 Output**     `(13, 13, 384)`


In [29]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Conv_3 = Conv2d((13, 13, 256), 384, (3, 3), 1, 1)

Output Size: (13, 13, 384)
Number of Parameters: 885120


## Layer 6 is  a Convolution Layer_4

- **Input**     `(13, 13, 384)`

- **Number of filters**     `384`

- **Filter size**     `(3, 3)`

- **Stride**     `1`

- **padding**     `1`

- **Layer 6 Output**     `(13, 13, 384)`

In [31]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Conv_4 = Conv2d((13, 13, 384), 384, (3, 3), 1, 1)

Output Size: (13, 13, 384)
Number of Parameters: 1327488


## Layer 7 is a Convolution Layer_5

- **Input**     `(13, 13, 384)`

- **Number of filters**     `256`

- **Filter size**     `(3, 3)`

- **Stride**     `1`

- **padding**     `1`

- **Layer 7 Output**     `(13, 13, 256)`

In [32]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Conv_5 = Conv2d((13, 13, 384), 256, (3, 3), 1, 1)
Conv_5

Output Size: (13, 13, 256)
Number of Parameters: 884992


## Layer 8 is a Max Pooling_3 Followed by Convolution_5

- **Input**     `(13, 13, 256)`

- **Max pooling**     `(3, 3)`  

- **Pooling size**(overlapping)     `1`

- **Stride**     `2`

- **Layer 8 Output**     `(6, 6, 256)`

In [34]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Max_pool_3 = MaxPool2d((13, 13, 256), (3, 3), 2)

Output Size: (6, 6, 256)
Number of Parameters: 0


## Layer 9 is a Fully_Connected layer_1

- **input**     `(6, 6, 256)`

- **flatten**     `6*6*256 = 9216`

- **output size**     `4096`

- **N** Number of input data      `9216`

In [36]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

FC1 = Linear(6*6*256, 4096)

Number of Parameters: 37752832


## Layer 10 is a Fully_Connected layer_2

- **input**     `4096`

- **output size**     `4096`

- **N** Number of input data =      `4096`

In [37]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

FC2 = Linear(4096, 4096)

Number of Parameters: 16781312


## Layer 11 is a Fully_Connected layer_3

- **input**     `4096`

- **output size**     `1000`

- **N** Number of input data =      `4096`

- **Num_classes** Number of labels =      `1000`

In [38]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

FC3 = Linear(4096, 1000)

Number of Parameters: 4097000


# CNNbasic Assignment#2

- 제출자: 20기 황태연
- 제출 일자: 2023.09.05. (화)

# AlexNet 구현

모델 구현 후 summary로 전체 모델 구조 출력과 주석으로 간단한 설명을 달아주시면 됩니다.

프레임워크는 자유이고, 기본 tensforflow와 pytorch tutorial 사이트를 아래에 첨부해 드립니다.

이 외 각 프레임워크 별 summary 등 추가적인 사용 방법은 구글링으로 찾아주세요!-!

- Tensorflow Tutorial: https://www.tensorflow.org/tutorials?hl=ko

- Pytorch Tutorial: https://tutorials.pytorch.kr/

<img src="01.png">

- 아래에서 구현할 모델을 다음 그림이 가장 잘 표현하고 있습니다.
<img src="07.png">

- AlexNet에는 Dropout이 구현되어 있지만, 과제에서 제시한 모델 구조에 맞도록 구현하기 위해 Dropout을 추가하지 않겠습니다.

## Pytorch

In [17]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

# device 설정 (cuda or cpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class AlexNet(nn.Module):
    def __init__(self, n_classes):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2))
        self.conv2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2))
        self.conv3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU())
        self.conv4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU())
        self.conv5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2))
        
        self.fc1 = nn.Sequential(
            nn.Linear(6*6*256, 4096),
            nn.ReLU())
        self.fc2 = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc3 = nn.Linear(4096, 1000)
            
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


# pytorch summary
model = AlexNet(1000).to(device)
summary(model, (3, 227, 227))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
            Conv2d-4          [-1, 256, 27, 27]         614,656
              ReLU-5          [-1, 256, 27, 27]               0
         MaxPool2d-6          [-1, 256, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         885,120
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]       1,327,488
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         884,992
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
           Linear-14                 [-